In [1]:
import numpy as np
import scipy.fftpack
import keras

Using TensorFlow backend.


In [2]:
(train_X, train_Y), (test_X, test_Y) = keras.datasets.mnist.load_data()
(neg_X, _), (_, _) = keras.datasets.fashion_mnist.load_data()
train_X = train_X[:,:,:,np.newaxis] / 255
test_X = test_X[:,:,:,np.newaxis] / 255
neg_X = neg_X[:,:,:,np.newaxis] / 255

In [3]:
X_freq = scipy.fftpack.dct(train_X, norm='ortho', axis=1)
X_freq = scipy.fftpack.dct(X_freq, norm='ortho', axis=2)
X_freq_mean = np.mean(X_freq, axis=0, keepdims=True)
X_freq_std = np.std(X_freq, axis=0, ddof=1, keepdims=True)

In [4]:
def generate_neg_samples():
    x = X_freq_mean + np.random.randn(*train_X.shape) * X_freq_std
    x = scipy.fftpack.idct(x, norm='ortho', axis=2)
    x = scipy.fftpack.idct(x, norm='ortho', axis=1)
    x = np.clip(x, 0, 1)
    return x

In [5]:
X = X_input = keras.layers.Input((28,28,1))
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Conv2D(8, (3,3), padding='same', kernel_initializer='he_uniform')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(8, (3,3), padding='same', kernel_initializer='he_uniform')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(8, (3,3), padding='same', kernel_initializer='he_uniform')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(11, activation='softmax')(X)
M = keras.Model(X_input, X)
M_opt = keras.optimizers.Nadam(lr=2e-6)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         80        
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         584       
__________

In [6]:
def datagen(batch_size):
    batch_size = batch_size // 2
    while True:
        shuf = np.arange(0, train_X.shape[0], 1, dtype=int)
        neg = generate_neg_samples()
        for i in range(int(np.ceil(train_X.shape[0] / batch_size))):
            train_X_ = train_X[shuf[i*batch_size:(i+1)*batch_size]]
            train_Y_ = train_Y[shuf[i*batch_size:(i+1)*batch_size]]
            yield train_X_, train_Y_
            neg_ = neg[i*batch_size:(i+1)*batch_size]
            yield neg_, np.full_like(train_Y_, 10, dtype=int)

In [7]:
M.fit_generator(datagen(64), (train_X.shape[0] // 64)*2, validation_data=(test_X, test_Y), epochs=10)

Epoch 1/10
1874/1874 [==============================] - 22s 12ms/step - loss: 0.1371 - acc: 0.9599 - val_loss: 0.0887 - val_acc: 0.9715
Epoch 2/10
1874/1874 [==============================] - 16s 9ms/step - loss: 0.0453 - acc: 0.9864 - val_loss: 0.1128 - val_acc: 0.9656
Epoch 3/10
1874/1874 [==============================] - 20s 11ms/step - loss: 0.0338 - acc: 0.9897 - val_loss: 0.0698 - val_acc: 0.9795
Epoch 4/10
1874/1874 [==============================] - 17s 9ms/step - loss: 0.0307 - acc: 0.9905 - val_loss: 0.0772 - val_acc: 0.9760
Epoch 5/10
1874/1874 [==============================] - 20s 11ms/step - loss: 0.0249 - acc: 0.9927 - val_loss: 0.0975 - val_acc: 0.9725
Epoch 6/10
1874/1874 [==============================] - 17s 9ms/step - loss: 0.0250 - acc: 0.9924 - val_loss: 0.0617 - val_acc: 0.9829
Epoch 7/10
1874/1874 [==============================] - 20s 11ms/step - loss: 0.0207 - acc: 0.9938 - val_loss: 0.0837 - val_acc: 0.9764
Epoch 8/10
1874/1874 [=============================

In [19]:
pred = np.argmax(M.predict(train_X), axis=-1)
np.sum(pred == 10) / train_X.shape[0]

3.3333333333333335e-05

In [20]:
pred = np.argmax(M.predict(test_X), axis=-1)
np.sum(pred == 10) / test_X.shape[0]

0.0002

In [21]:
pred = np.argmax(M.predict(neg_X), axis=-1)
np.sum(pred == 10) / neg_X.shape[0]

0.3204